In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gekko import GEKKO
from scipy.signal import savgol_filter

In [2]:
czantoria_full = pd.read_csv('data/csv/czantoria_sauce.csv')
czantoria_full.columns = ['time', 'distance', 'heartrate', 'cadence', 'velocity_smooth', 'altitude', 'grade_smooth']
czantoria_nonzero = czantoria_full[(czantoria_full['velocity_smooth'] > 0) | (czantoria_full['time'] == 0)]
czantoria_time = czantoria_nonzero.groupby('distance').agg({'time': 'max'}).reset_index()['time']
czantoria_final = pd.merge(czantoria_nonzero, czantoria_time, on=['time'])

distance_data = czantoria_final['distance'].to_numpy()
grade_data = czantoria_final['grade_smooth'].to_numpy()
elevation_data = czantoria_final['altitude'].to_numpy()

grade_smooth = savgol_filter(grade_data, 501, 1)

grade_data = grade_smooth

print(len(grade_data))

n = distance_data.shape[0]
to_stay = np.zeros(n)
to_stay[0] = 1

for i in range(n - 1):
    if np.abs(grade_data[i+1] - grade_data[i]) < 1.1:
        grade_data[i+1] = grade_data[i]
        to_stay[i+1] = 0
    elif np.sign(grade_data[i+1]) != np.sign(grade_data[i]):
        to_stay[i + 1] = 1
    else:
        to_stay[i + 1] = 1
        
to_stay[-1] = 1

grade_data = grade_data[to_stay == 1]
distance_data = distance_data[to_stay == 1]
elevation_data = elevation_data[to_stay == 1]

print(len(grade_data))

grade_data = grade_data/100

distance = distance_data[-1]
t_est = 3 * 60 * 60

x_data = distance_data
slope_data = grade_data

13385
454


In [3]:
m = GEKKO()

# time points [s]
nt = 201
tm = np.linspace(0,1,nt)
m.time = tm

t_est = 4 * 60 * 60
mass = 55
sigma = 20
E0 = 1400
c = 3.4
g = 9.81
i = 0.4
rho = 1.205
cdA = 0.24
v_w = 0
eff = 1
P_max = 22.6

# MV
P = m.MV(value=0, lb=0, ub=1)
P.STATUS = 1

# Variables
x = m.Var(value=0, lb=0)
E = m.Var(value=E0, lb=0)
v = m.Var(value=0, lb=0)
slope = m.Var(value=slope_data[0])
# slope = 0

# slope
m.cspline(x, slope, x_data, slope_data, True)

Cr = m.Intermediate(39.93 * slope**2 + 16.81 * slope + 3.63)
alpha = m.Intermediate(m.atan(m.abs3(slope)))

p = np.zeros(nt)
p[-1] = 1.0
final = m.Param(value=p)

# FV - final time
tf = m.FV(value=t_est, lb = 0.1)
tf.STATUS = 1


m.Equation(E.dt() == (sigma - P * P_max) * tf)
m.Equation(x.dt() == v * m.cos(alpha) * tf)
m.Equation(P * P_max == Cr * v + 0.5 * rho * cdA * v**3 + g * v * m.sin(alpha))
m.Equation((distance - x)*final<=0)
m.Equation(sigma - P * P_max <= 0)

m.Minimize(tf)

m.options.IMODE = 6
m.options.MAX_ITER=50000
m.options.SOLVER=3

m.options.RTOL = 10**(-2)
m.options.OTOL = 10**(-2)

m.solve()

print('Final Time: ' + str(tf.value[0]))

apm 90.156.80.122_gk_model0 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            1
   Constants    :            0
   Variables    :           13
   Intermediates:            2
   Connections  :            2
   Equations    :           11
   Residuals    :            9
 
 Number of state variables:           3401
 Number of total equations: -         3000
 Number of slack variables: -          800
 ---------------------------------------
 Degrees of freedom       :           -399
 
 * Warning: DOF <= 0
 **********************************************
 Dynamic Control with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This 

  61  2.8774830e+06 2.71e+05 4.10e+13  12.0 9.56e+07    -  2.83e-06 4.88e-06f  1
  62  2.8779233e+06 2.71e+05 4.11e+13  12.0 1.15e+07   4.3 4.75e-05 6.70e-05f  1
  63  2.8774466e+06 2.71e+05 4.11e+13   4.0 2.52e+07    -  2.76e-05 4.30e-05f  1
  64  2.8748075e+06 2.71e+05 4.19e+13   4.0 2.14e+07    -  2.72e-06 3.10e-04f  1
  65  2.8746673e+06 2.70e+05 4.18e+13   4.0 7.82e+06   4.7 5.46e-05 1.50e-04h  1
Reallocating memory for MA57: lfact (8102528)
  66  2.8744755e+06 2.70e+05 4.17e+13   4.0 1.32e+05   6.9 3.44e-04 1.10e-04h  1
Reallocating memory for MA57: lfact (8531432)
  67  2.8743574e+06 2.70e+05 4.17e+13  11.5 2.62e+04   8.2 1.66e-04 7.51e-05h  1
  68  2.8741774e+06 2.70e+05 4.15e+13  12.0 1.17e+05   7.8 8.50e-05 1.07e-04h  1
  69  2.8739886e+06 2.70e+05 4.03e+13  12.0 1.02e+06   7.3 4.96e-05 9.26e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70r 2.8739886e+06 2.70e+05 1.00e+03  12.0 0.00e+00   6.8 0.00e+00 4.24e-07R  4
  71r 2.8771369e+

 153  1.3004891e+06 9.70e+04 7.99e+14   6.0 2.16e+06   5.3 4.89e-04 1.04e-03f  1
 154  1.3006299e+06 9.70e+04 7.99e+14   6.0 6.23e+06    -  1.32e-04 1.53e-05h  1
 155  1.3001317e+06 9.67e+04 8.05e+14   6.0 2.88e+05   5.7 1.67e-03 2.59e-03h  1
 156  1.3012801e+06 9.63e+04 8.67e+14  12.2 8.18e+05   5.2 1.27e-03 4.83e-03f  1
 157  1.2969088e+06 9.56e+04 1.08e+15  13.1 2.96e+05   5.6 2.09e-02 7.34e-03h  1
 158  1.2970651e+06 9.51e+04 1.16e+15  13.5 8.62e+05    -  8.48e-03 4.80e-03f  1
 159  1.2998435e+06 9.49e+04 1.05e+15  13.8 2.76e+06    -  3.10e-03 2.37e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160  1.2980788e+06 9.46e+04 1.09e+15   6.2 3.52e+05   6.1 5.48e-03 3.02e-03h  1
 161  1.3008775e+06 9.45e+04 1.05e+15   6.2 2.59e+06    -  1.98e-05 1.27e-03f  1
 162  1.2998217e+06 9.43e+04 1.07e+15  13.7 3.59e+05   5.6 1.80e-02 2.06e-03f  1
 163  1.3007307e+06 9.40e+04 1.01e+15   6.4 1.17e+06    -  4.77e-04 2.70e-03f  1
 164  1.3006845e+06 9.40e+04

 246  7.7538721e+05 1.29e+05 2.95e+13   5.8 1.17e+04  11.6 9.90e-01 3.16e-03h  1
 247  7.7538348e+05 1.29e+05 2.95e+13   9.0 1.17e+04  12.0 1.00e+00 2.89e-05h  1
 248  7.7540632e+05 1.29e+05 2.96e+13  11.0 1.17e+04  11.6 1.84e-03 6.01e-05h  1
 249  7.7568209e+05 1.29e+05 2.95e+13  11.1 1.17e+04  11.1 7.63e-03 6.97e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  7.7574248e+05 1.29e+05 5.41e+14  11.1 1.17e+04  10.6 1.00e+00 1.54e-04h  1
 251  7.7890769e+05 1.28e+05 4.88e+14  13.0 1.17e+04  10.1 4.07e-01 9.12e-03h  1
 252  7.7566381e+05 1.26e+05 4.80e+14   6.7 4.77e+05    -  1.51e-02 1.80e-02h  1
 253  7.7196464e+05 1.24e+05 4.63e+14   6.7 4.00e+05    -  3.99e-02 1.87e-02h  1
 254  7.7191006e+05 1.23e+05 4.56e+14   6.6 1.11e+05    -  1.37e-02 5.05e-03h  1
 255  7.7108920e+05 1.20e+05 4.39e+14  12.6 4.10e+05    -  3.59e-02 2.47e-02h  1
 256  7.5922367e+05 1.13e+05 4.22e+14   6.6 1.90e+05    -  3.67e-02 5.48e-02h  1
 257  7.5921738e+05 1.13e+05

 @error: Solution Not Found


Exception:  @error: Solution Not Found


In [ ]:
tf.value[0]/3600

In [ ]:
import seaborn as sn

plt.rcParams['figure.figsize'] = [12, 8]
sn.set_context("talk")

plt.plot(x.value, v.value, label=r'$v$')
plt.plot(x.value, slope.value, label='slope')
plt.xlabel(r'distance $[m]$')
plt.ylabel(r'v $[m/s]$')
plt.title("Velocity strategy - route 1")
plt.legend()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
sn.set_context("talk")

plt.plot(x.value, P_max * np.array(P.value), label=r'$v$')
plt.plot(x.value, slope.value, label='slope')
plt.xlabel(r'distance $[m]$')
plt.ylabel(r'P $[W/kg]$')
plt.title("Power - route 1")
plt.legend()
plt.show()

In [ ]:
plt.plot(x.value, np.array(P.value))
plt.plot(x.value, np.array(slope.value)/max(slope.value))

In [ ]:
plt.plot(x.value, E.value)